In [1]:
import pandas as pd
import numpy as np
import matlab.engine
import os
import sys
from tqdm import tqdm

In [2]:
def extract_fit_info(mrs_fit):
    gaba = mrs_fit["GABA"]

    gaba_linewidth = float(gaba["FWHM"])
    gaba_snr = float(gaba["SNR"])
    gaba_fit_error = float(gaba["FitError"])
    gaba_conc_water = float(gaba["ConcIU"])
    gaba_conc_cr = float(gaba["ConcCr"])

    glx = mrs_fit["Glx"]

    glx_linewidth = float(glx["FWHM"])
    glx_snr = float(glx["SNR"])
    glx_fit_error = float(glx["FitError"])
    glx_conc_water = float(glx["ConcIU"])
    glx_conc_cr = float(glx["ConcCr"])

    return {
        "gaba_snr":gaba_snr,
        "gaba_linewidth":gaba_linewidth,
        "gaba_fit_error":gaba_fit_error,
        "gaba_conc_water":gaba_conc_water,
        "gaba_conc_cr":gaba_conc_cr,
        "glx_snr":glx_snr,
        "glx_linewidth":glx_linewidth,
        "glx_fit_error":glx_fit_error,
        "glx_conc_water":glx_conc_water,
        "glx_conc_cr":glx_conc_cr,
    }

In [5]:
eng = matlab.engine.start_matlab()

In [7]:
read_folder = "data/gannet_acc_fit_size_increases"
filenames = os.listdir(read_folder)

pre_df = []
for filename in filenames:
    
    range_start = filename.split(".")[0].split("_")[-2]
    range_end = filename.split(".")[0].split("_")[-1].split(".")[0]

    file_id = filename[:6]
    if file_id[:3]=="p10":
        file_id=filename[:7]
    #
    mrs_fit = eng.load(f"{read_folder}/{filename}")["mrs_fit"]["vox1"]
    pre_row = {"filename":file_id,"size":int(int(range_end)-int(range_start)+1)}
    mrs_fit_info = extract_fit_info(mrs_fit)
    pre_row.update(mrs_fit_info)
    pre_df.append(pre_row)


    #break
df = pd.DataFrame(pre_df)
#print(df)

In [8]:
df.to_csv("eval/fit_ranges/size_increase.csv",index=False)

In [16]:
df.shape[0]/1

(3600, 12)

In [50]:
df.groupby("filename").agg({"gaba_conc_water":"std","gaba_conc_cr":"std"}).median()

gaba_conc_water    0.178824
gaba_conc_cr       0.010159
dtype: float64